In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from __future__ import division
import numpy as np
from setup import *
from OrbitalMotion import numerical_orbital_evolution, analytical_orbital_evolution, analytical_split
c,G,Msolar,AU, pc = univeral_constants()

RecursionError: maximum recursion depth exceeded during compilation

In [ ]:
#Choose system parameters 

#---Inner Binary---
m0 = 30*Msolar
m1 = 30*Msolar
f1 = 1e-3 #orbital frequency
a1 = semi_major_axis(m0+m1,f1)
e1 = 0.6 #initial eccentricity
g1 = np.pi/4
J1 = angular_momentum(m0,m1,e1,a1)#angular momentum. Eq. 12 of Randal

#---Outer Binary
beta=5
a2 = beta * a1
m2 = 10*Msolar
e2 = 0.6

#---Shared---
I = 60 * np.pi/180.0 #Inclination
nmodes = 20 #number of modes to sum over for GW
iota = 20 * np.pi/180  #angle of L vector w.r.t observer LOS 
D = 1e6 * pc #distance from source to observer 1Mpc


#---Integration settings
Tint = 1.0
fs = 1e-1
run = 1

#---Info for user
status(fs, Tint)


In [ ]:

import time 

if run == 1:

    #The full numerical solution at high resolution 
    output = numerical_orbital_evolution(m0,m1,m2,a1,e1,g1,J1,e2,a2,I,fs,Tint)
    numerical_motion_highres = output[0]
    constants = output[1]
    np.save('numerical_refined',numerical_motion_highres)

    
    
   
    
    
    
tstart = time.time()
#and the training set at low resolution
output = numerical_orbital_evolution(m0,m1,m2,a1,e1,g1,J1,e2,a2,I,1e-3,Tint)
numerical_motion_lowres = output[0]
constants = output[1]
np.save('numerical_coarse',numerical_motion_lowres)
tend = time.time()

print ('Coarse resolution ran in',tend-tstart,' seconds')


In [ ]:
import sys

#temp plotting setup
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(24,10)) 
ax1 = plt.subplot2grid((1,1), (0,0))
#end temp plottign setop


#Load the high resolution data
full_set  = np.load('numerical_refined.npy')

#Load the low resolution data
training_set = np.load('numerical_coarse.npy')

#Create an array to save the outputs
analytical = np.empty_like(full_set)





#Split numerical coarse into N training sets
N = 10
T_seconds = Tint*365*24*3600
t = np.arange(0,T_seconds,1/fs)
Tlim = T_seconds / N



#the time of each training set
tTRAIN = training_set[:,0]
tNUM = full_set[:,0]

#Get the incidence in the coarse data closest to the split point
idx = min(range(len(tTRAIN)), key=lambda i: abs(tTRAIN[i]-Tlim))
Tlim = tTRAIN[idx]


#Get the same point in the numerical data
idx2 = np.where(tNUM == Tlim)[0][0]


tstart = time.time()


for i in range(1):
    print ('-------', i, '--------')
  
    #Slice the training set data
    subset = training_set[i*idx : i*idx + (idx+1), :]

    #Slice the high resolution time
    tHR = full_set[i*idx2 : i*idx2 + (idx2+1),0]
    

    #Use this training set data to get a semi-analytical solution over the relevant time interval
    output = analytical_split(subset,tHR,constants)

    #Are the endpoints the same?
    w = 1
    ax1.plot(output[:,0], output[:,w])
    ax1.plot(subset[:,0], subset[:,w])

    tN = full_set[i*idx2 : i*idx2 + (idx2+1),0] #this is the same as t subset
    gN = full_set[i*idx2 : i*idx2 + (idx2+1),w]
    
    
    print ('E = ', output[0,0], tHR[0],tN[0])
    print ('E = ', output[-1,0], tHR[-1],tN[-1])
    



   
    #ax1.plot(tN,gN)
    #ax1.plot(tN, (output[:,w] - gN)/gN)
      
    if i == 0:
        np.save('EccentricityData/edata_e='+str(e1)+'.npy', subset)
        
        togo = np.zeros((len(tN),2))
        togo[:,0] = tN
        togo[:,1] = gN
        
        np.save('EccentricityData/edataHR_e='+str(e1)+'.npy', togo)
        
    
    analytical[i*idx2 : i*idx2 + (idx2+1),:] = output

        
tend = time.time()

print ('Total runtime = ', tend-tstart)

lim = 0.025
#ax1.set_ylim(-0.0125,0.0075)

In [ ]:

from plot import *
### Now plot and compare
#plot_motion(analytical)
#plot_motion(full_set)
plot_compare_motion(analytical,full_set)


In [ ]:
import GravRadiation as GR
constants = np.array([m0+m1, nmodes,iota,m0*m1/(m0+m1),D])
waveformNR = GR.Gwaves(full_set, constants)
waveformAN = GR.Gwaves(analytical, constants)

In [ ]:
#Plot the GW waveforms
from plot import *
plot_GW(waveformNR,f1)
plot_GW(waveformAN,f1)
compare_GW(waveformNR,waveformAN)

In [ ]:
#Go to frequency domain and get the overlap
f,h1,h2, S = GR.overlap(waveformNR,waveformAN)
plot_GW_frequency(f,h1,h2, S)